In [1]:
import pandas as pd

In [2]:
ment = pd.read_csv("Shodhganga_mentorship.csv", sep=',')

In [3]:
ment['instituteId'].fillna("I000",inplace=True)
ment['dc.date.submitted[]'].fillna(value=ment['dc.date.awarded'],inplace=True)
ment['DepartmentId'].fillna("D000",inplace=True)
#ment['dc.subject.ddc']= ment['dc.subject.ddc'].fillna(value=ment['dc.publisher.department[]'])
#ment['dc.publisher.department[]']= ment['dc.publisher.department[]'].fillna(value=ment['dc.publisher.institution[]'])
ment['dc.subject.ddc']= ment['dc.subject.ddc'].replace(r"\|?\d+::",",", regex=True).str.strip(",")
ment['dc.date.submitted[]'] = pd.to_datetime(ment['dc.date.submitted[]'],errors = 'coerce')

In [4]:
ment.columns

Index(['advisorId', 'researcherId', 'advId_1', 'studId_1', 'advisor_name',
       'researcher_name', 'dc.date.submitted[]', 'dc.language.iso[]',
       'dc.publisher.department[]', 'DepartmentId',
       'dc.publisher.institution[]', 'instituteId', 'dc.title[]', 'thesisId',
       'dc.type.degree', 'dc.date.awarded', 'dc.subject.ddc', 'dc.source.uri',
       'dc.description.abstract', 'dc.title.alternative'],
      dtype='object')

In [5]:
ment['advisor_inst_dept']=ment['advisor_name']+"@"+ment['instituteId']+"@"+ment['DepartmentId']
ment['stud_inst_dept']=ment['researcher_name']+"@"+ment['instituteId']+"@"+ment['DepartmentId']
index1=pd.unique(ment[['advisor_inst_dept', 'stud_inst_dept']].values.ravel('K'))
name_index=pd.DataFrame({'names_inst_dept':index1})
name_index['rid']=name_index.index
ment["advisorId"]=ment['advisor_inst_dept'].map(name_index.set_index('names_inst_dept')['rid'])
ment["researcherId"]=ment['stud_inst_dept'].map(name_index.set_index('names_inst_dept')['rid'])

In [ ]:
#merge_result= merge_result.groupby('pid')['institute'].apply(list).reset_index(name='inst')

In [14]:
advisor_detail=ment.groupby(['advisorId','advisor_name','instituteId',
                               'DepartmentId'], as_index=False)[['dc.publisher.institution[]','dc.publisher.department[]','dc.date.submitted[]','dc.title[]','dc.subject.ddc']].agg(lambda x: list(x))

In [18]:
advisor_detail.head()

,advisorId,advisor_name,instituteId,DepartmentId,dc.publisher.institution[],dc.publisher.department[],dc.date.submitted[],dc.title[],dc.subject.ddc
0,0,"Jose, Joseph",I251,D1321,Mahatma Gandhi University,department of sacred heart college(thevara). d...,[1998-01-01 00:00:00],[Chromosome studies and exploration of chemica...,"[Natural sciences & mathematics,Plants (Botany)]"
1,1,"Raju, K. V.",I251,D2490,Mahatma Gandhi University,department of sacred heart college,"[2015-05-29 00:00:00, 2013-08-04 00:00:00, 200...",[Fishing industry in Kerala The perils of inte...,"[Social sciences, Social sciences, Social scie..."
2,2,"Joseph, K. K.",I251,D2490,Mahatma Gandhi University,department of sacred heart college,[2015-08-24 00:00:00],[Savings and investment pattern of stock inves...,[Social sciences]
3,3,"Thomas, E. M.",I251,D2490,Mahatma Gandhi University,department of sacred heart college,"[2015-07-15 00:00:00, 2015-11-12 00:00:00]",[An economic analysis of Kole cultivation in K...,"[Social sciences, Social sciences]"
4,4,"Francis, M. S.",I251,D2490,Mahatma Gandhi University,department of sacred heart college,"[2015-12-28 00:00:00, 2015-12-28 00:00:00, 200...",[Phytochemical and Molecular Systematics of Ar...,"[Social sciences, Social sciences, Social scie..."


In [15]:
advisor_detail['dc.publisher.institution[]']=advisor_detail['dc.publisher.institution[]'].apply(lambda x: x[0])

In [16]:
advisor_detail['dc.publisher.department[]']=advisor_detail['dc.publisher.department[]'].apply(lambda x:x[0])

In [20]:
#advisor_detail.iloc[50:100]['dc.publisher.institution[]']